# Capstone Segmentation and Clustering of Toronto Neighborhoods

<hr></hr>

This workbook will satisfy the week 3 requirements for the IBM Data Science Capstone course on Coursera. In it, I will scrape tabular data from wikipedia, read it into a pandas dataframe, then clean that data. First, I will need to import the BeautifulSoup library to be able to read this data into a dataframe.

### Step 1: Scrape the Website

In [5]:
import pandas as pd
import requests

In [4]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [2]:
from bs4 import BeautifulSoup

In [3]:
BeautifulSoup

bs4.BeautifulSoup

In [6]:
r1 = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(r1.content,'html.parser')
table = soup.find(lambda tag: tag.name =='table' and ("wikitable" in tag['class']))

In [7]:
df = pd.read_html(str(table), flavor='bs4')[0]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Step 2: Remove the "Not Assigned" data
Do this by creating a dataframe where it selects those that are not assigned, then inverse it.

In [8]:
df_2 = (df['Borough'] == 'Not assigned')|(df['Neighbourhood'] == 'Not assigned')
df = df[~df_2]

### Step 3: Check to see if there are any "Not Assigned" variables left

In [9]:
(df.Borough == 'Not assigned').sum()

0

In [10]:
(df.Neighbourhood == 'Not Assigned').sum()

0

In [11]:
post=df.Postcode.unique()

### Step 4: Group the dataframe by postcode
Some entries have the same postcode, so we need to group them together based on the unique postcode using the variable above.

In [12]:
Toronto = pd.DataFrame(columns=['Postcode','Borough','Neighbourhood'])
for code in post:
    temp_df = df[['Borough','Neighbourhood']][df['Postcode'] == code]
    boro = temp_df.Borough.unique()
    hood = temp_df.Neighbourhood.unique()
    Toronto = Toronto.append({
        'Postcode':code,
        'Borough':",".join(boro),
        'Neighbourhood':",".join(hood)},ignore_index=True)

In [13]:
Toronto.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M9A,Etobicoke,Islington Avenue


In [14]:
Toronto.shape

(102, 3)

### Step 5: Install the Geocoder to Geocode the Post Codes
#### Note: This did not work so I got the data from the CSV

In [15]:
#!conda install -c conda-forge geocoder --yes

Solving environment: - ^C
failed

CondaError: KeyboardInterrupt



In [25]:
#import geocoder # import geocoder

# initialize your variable to None
#lat_lng_coords = None

# loop until you get the coordinates
#while(lat_lng_coords is None):
  #g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  #lat_lng_coords = g.latlng

#latitude = lat_lng_coords[0]
#longitude = lat_lng_coords[1]

ModuleNotFoundError: No module named 'geocoder'

In [16]:
postalcodes_from_csv = pd.read_csv('http://cocl.us/Geospatial_data')

In [18]:
postalcodes_from_csv.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [19]:
Toronto = Toronto.sort_values(by=['Postcode'])
Toronto.reset_index(inplace=True, drop=True)

In [20]:
pcode = postalcodes_from_csv.sort_values(by=['Postal Code'])

In [21]:
pcode.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [22]:
Toronto=pd.concat([Toronto, pcode[['Latitude','Longitude']]], axis = 1)

In [23]:
Toronto.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
